In [39]:
import pandas as pd
import os
from pymongo import MongoClient
from collections import defaultdict
from urllib.parse import quote_plus

In [6]:
print(os.getcwd())

C:\Users\jalop\Documents\001-TFM\tfm-bienvenida\TFM_Repo\notebook


In [9]:
print(os.listdir())

['.ipynb_checkpoints', 'catalogo_datos.ipynb', 'datos_poblacion.ipynb', 'diccionario_datos_mongo.xlsx', 'Untitled.ipynb', '~$diccionario_datos_mongo.xlsx']


In [22]:
dtype_map = {
    'Periodo':   'int64',     # o 'Int64' si quieres permitir NaN
    'Edad':      'string',    # para capturar valores como “Todas las edades”
    'Sexo':      'category',
    'Escenario': 'category',
    'Total':     'int64'      # o 'int64' si no hay nulos
}

df_raw = pd.read_csv(
    '../data/36652.csv',
    sep=';',
    decimal=',',
    encoding='utf-8'
)

In [23]:
df_raw.sample()

,Escenario,Sexo,Edad,Periodo,Total
11986,Central,Mujeres,30,2073,301440.89


In [24]:
print(df_raw.dtypes)

Escenario     object
Sexo          object
Edad          object
Periodo        int64
Total        float64
dtype: object


In [25]:
# Quitamos el agregado de “Todas las edades”
df_raw = df_raw[df_raw['Edad'] != 'Todas las edades'].copy()

# Convertir Edad a entero
df_raw['Edad'] = df_raw['Edad'].astype(int)

In [26]:
print(df_raw.dtypes)

Escenario     object
Sexo          object
Edad           int64
Periodo        int64
Total        float64
dtype: object


In [27]:
df_raw.sample()

,Escenario,Sexo,Edad,Periodo,Total
69870,Saldo migratorio bajo,Hombres,43,2074,301338.41


In [28]:
df_raw['Edad'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100])

In [29]:
df_raw['Escenario'].unique()

array(['Central', 'Fecundidad alta', 'Fecundidad baja',
       'Saldo migratorio alto', 'Saldo migratorio bajo',
       'Fecundidad y saldo migratorio altos',
       'Fecundidad y saldo migratorio bajos', 'Saldo migratorio nulo'],
      dtype=object)

In [30]:
df_raw['Periodo'].unique()

array([2074, 2073, 2072, 2071, 2070, 2069, 2068, 2067, 2066, 2065, 2064,
       2063, 2062, 2061, 2060, 2059, 2058, 2057, 2056, 2055, 2054, 2053,
       2052, 2051, 2050, 2049, 2048, 2047, 2046, 2045, 2044, 2043, 2042,
       2041, 2040, 2039, 2038, 2037, 2036, 2035, 2034, 2033, 2032, 2031,
       2030, 2029, 2028, 2027, 2026, 2025, 2024])

In [32]:
df_raw['Sexo'].unique()

array(['Total', 'Hombres', 'Mujeres'], dtype=object)

In [34]:
df_raw = df_raw.rename(columns={
    'Periodo': 'year',
    'Edad': 'age',
    'Sexo': 'sex',
    'Escenario': 'scenario',
    'Total': 'population'
})

In [35]:
df_raw.sample()

,scenario,sex,age,year,population
16892,Fecundidad alta,Total,24,2063,640018.96


In [36]:
sex_map = {
    'Total':   'T',   
    'Hombres': 'M',   
    'Mujeres': 'F'   
}

df_raw['sex'] = df_raw['sex'].map(sex_map).astype('category')

In [37]:
df_raw.sample()

,scenario,sex,age,year,population
55058,Saldo migratorio alto,M,58,2045,352005.26


In [40]:
# Configurar conexión
usuario = "jalope"
contrasena = "admin"
host = "127.0.0.1"
puerto = "27250"

uri = f"mongodb://{quote_plus(usuario)}:{quote_plus(contrasena)}@{host}:{puerto}/?directConnection=true"
client = MongoClient(uri)
db = client["tfm_db"]

In [42]:
coll = db['INE_36652_POBLACION_PROJ_RAW']
coll.drop()
coll.insert_many(df_raw.to_dict('records'))
print("Registros en la colección: ", coll.count_documents({}))
print("Número de filas de df_raw: ", len(df_raw))

Registros en la colección:  123624
Número de filas de df_raw:  123624
